In [1]:
import os
import sys
from pprint import pp
from collections import Counter

from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
from pymongo import MongoClient, UpdateOne

client = MongoClient("mongodb://localhost:27017/")
db = client["podcast_db"]  # Database name
episodes_collection = db["episodes"]  # Collection for episodes
search_index_collection = db["search_index"]  # Collection for full-text search

In [24]:
EPISODE_FILES = [str(f) for f in d.glob('EP6*.txt')] 
                #[str(f) for f in d.glob('EP2*.txt')] + \
                #[str(f) for f in d.glob('EP3*.txt')]
EPISODE_FILES = sorted(EPISODE_FILES)
pp(EPISODE_FILES)

#EPISODE_FILES = [
#    '../transcripts/EP60_1 面試者：噢我的缺點就是真的太瘦了～.txt',
#    '../transcripts/EP60_2 對不起我洗完臉沒把毛巾擰乾我真該死嗚嗚嗚.txt',
#    '../transcripts/EP61 我是國小潔牙小隊長也要告訴你嗎!?.txt',
#    '../transcripts/EP62 不要再玩詐騙手遊了！.txt',
#    '../transcripts/EP63 生命中有太多大便了！.txt',
#    '../transcripts/EP64 以前的MSN狀態真的太恥啊啊啊啊！本集攝影師老婆代班.txt',
#    '../transcripts/EP65 FB廢片的正確使用方式！？.txt',
#    '../transcripts/EP66 吸管跟免洗筷最討厭了哼！.txt',
#    '../transcripts/EP67 跟阿公去了超奇幻遊覽車進香團？.txt',
#    '../transcripts/EP68 漂泊的童年與吃餿水的故事!?!?｜脆脆漂泊童年第１集.txt',
#    '../transcripts/EP69 我是傳說中的桃園天才小學生｜脆脆漂泊童年第２集.txt',
#]

['../tokenized_tw_transcripts/EP6 罰站就是要背99乘法表！.txt',
 '../tokenized_tw_transcripts/EP60_1 面試者：噢我的缺點就是真的太瘦了～.txt',
 '../tokenized_tw_transcripts/EP60_2 對不起我洗完臉沒把毛巾擰乾我真該死嗚嗚嗚.txt',
 '../tokenized_tw_transcripts/EP61 我是國小潔牙小隊長也要告訴你嗎!?.txt',
 '../tokenized_tw_transcripts/EP62 不要再玩詐騙手遊了！.txt',
 '../tokenized_tw_transcripts/EP63 生命中有太多大便了！.txt',
 '../tokenized_tw_transcripts/EP64 以前的MSN狀態真的太恥啊啊啊啊！本集攝影師老婆代班.txt',
 '../tokenized_tw_transcripts/EP65 FB廢片的正確使用方式！？.txt',
 '../tokenized_tw_transcripts/EP66 吸管跟免洗筷最討厭了哼！.txt',
 '../tokenized_tw_transcripts/EP67 跟阿公去了超奇幻遊覽車進香團？.txt',
 '../tokenized_tw_transcripts/EP68 漂泊的童年與吃餿水的故事!?!?｜脆脆漂泊童年第１集.txt',
 '../tokenized_tw_transcripts/EP69 我是傳說中的桃園天才小學生｜脆脆漂泊童年第２集.txt']


In [33]:
episodes = []

for episode_file in EPISODE_FILES:
    episode_name = episode_file[episode_file.rfind('/')+1:-4]
    with open(episode_file, 'r') as f:
        sentences = f.read().strip().split('\n')
    tokenized_sentences = [[word for word in sentence.split(' ') if word] for sentence in sentences]
    tokenized_words = [word for sentence in tokenized_sentences for word in sentence]
    word_set = set(tokenized_words)

    ep_num_part = episode_name[2:episode_name.find(' ')]
    if ep_num_part.find('_') != -1:
        ep_num = int(ep_num_part[:ep_num_part.find('_')])
        ep_part = int(ep_num_part[ep_num_part.find('_')+1:])
    else:
        ep_num = int(ep_num_part)
        ep_part = None

    episode = {
        'ep_num': ep_num,
        'ep_part': ep_part,
        'sentences': tokenized_sentences,
        'words': tokenized_words,
        'word_set': word_set
    }
    episodes.append(episode)
    print(episode_name, 'done')
    
print(episodes[0]['sentences'][:10])
print(episodes[0]['words'][:30])
#print(episodes[0]['word_set'])

EP6 罰站就是要背99乘法表！ done
EP60_1 面試者：噢我的缺點就是真的太瘦了～ done
EP60_2 對不起我洗完臉沒把毛巾擰乾我真該死嗚嗚嗚 done
EP61 我是國小潔牙小隊長也要告訴你嗎!? done
EP62 不要再玩詐騙手遊了！ done
EP63 生命中有太多大便了！ done
EP64 以前的MSN狀態真的太恥啊啊啊啊！本集攝影師老婆代班 done
EP65 FB廢片的正確使用方式！？ done
EP66 吸管跟免洗筷最討厭了哼！ done
EP67 跟阿公去了超奇幻遊覽車進香團？ done
EP68 漂泊的童年與吃餿水的故事!?!?｜脆脆漂泊童年第１集 done
EP69 我是傳說中的桃園天才小學生｜脆脆漂泊童年第２集 done
[['嗨', '大家', '好', '我', '是', '翠翠', '歡迎', '來到', '好味', '小姐', '開束', 'wszystk', '我', '要', '還', '你', '原型', '沒有', '要', '對不起'], ['沒有', '要', '我', '還', '以為', '你', '終於', '念對', '了'], ['我', '剛剛', '本來', '有', '一點', '小', '得意', '的'], ['那', '你', '再', '念', '一', '次', '好了'], ['好味', '小姐', '開束', 'wszystk', '我', '還', '你', '原型', '好', '可以', '可以'], ['喔', '唸完', '了'], ['我', '發現', '你們', '前面', '都', '忘記', '自我', '介紹', '了'], ['那', '自我', '介紹', '就', '被', '你', '打斷', '了'], ['沒有', '錯', '就是', '為什麼', '我', '會', '沒有', '自我', '介紹'], ['而且', '然後', '人家', '在', '開頭', '之前', '就', '先', '自我', '介紹']]
['嗨', '大家', '好', '我', '是', '翠翠', '歡迎', '來到', '好味', '小姐', '開束', 'wszystk', '我', '要', '還', '你', '原型', '沒有', '要', 

In [34]:
s = set()
for ep in episodes:
    s.update(ep['word_set'])
    
print('Total words:', sum([len(ep['words']) for ep in episodes]))
print('Unique words:', len(s))

Total words: 186203
Unique words: 11530


### Build Stopwords List

In [35]:
stopwords = [
    '的', '你', '我', '他', '她', '它', '牠', '就', '是', '個', '了', '這', '那', '對', '有', '一', '不', '啊', '在', '說',
    '很','都','喔','要','會','嗎','好','人','跟','問','欸','嗯','也','來','還','隻','看','把','小','去','被','講','種', '太',
    '到', '吧', '沒', '嘿', '做', '上', '啦', '嘛', '又', '大', '先', '哦', '耶', '像', '給', '下', '幫', '兩', '得', '再',
    '呢', '次', '想', '多', '錯', '讓', '每', '誰', '幾', '最','從','四','著','話','差','聊','者','整','點','全','才','事',
    '片','五','坐','過','舉','份','怕','但','裡','蠻','超','只','能', '天', '用', '三', '件', '中', '誒',
    # 名詞
    '時候', '東西', '大家', '事情', '我們', '你們', '他們', '牠們','現在', '裡面', '外面', '自己', '感覺', '今天', '地方',
    # 代名詞
    '這樣', '那樣', '這邊', '那邊','什麼',
    # 動詞
    '覺得', '想說','知道', '開始','看到', '比較', '記得', '出來',
    # 形容詞
    '一些', '沒有','可以','大概', '之類', '一下', '最後', '不會','不錯', '很多', '突然', '真的', '一定', '直接', '非常', '如果', '一樣',
    '一點', '特別', '最近', '以前', 
    # 副詞
    '真的', '就是', '可能','已經','之前','之後', '一直', '好像', '不要', '不行', '後來', '有點', '還是', '應該','不然', '結果',
    # 連接詞
    '然後', '所以', '因為', '但是', '其實', '反正', '而且', '或是',
    # 常見詞
    '為什麼', '怎麼', '這麼', '那麼', '這樣子', '幹嘛',

    # 其他
    '留言',
]

filtered_words = [word for ep in episodes for word in ep['words'] if word not in stopwords and len(word) == 3]
word_counts = Counter(filtered_words)

#pp(word_counts.most_common())

### Find Topics

In [44]:
# TF-IDF
sentences = [' '.join([word for word in ep['words'] if word not in stopwords]) for ep in episodes]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Define how many top keywords you want per episode
top_n = 6

# Iterate over each document (episode)
for doc_idx in range(tfidf_matrix.shape[0]):
    # Get the row for the document in sparse format
    doc = tfidf_matrix[doc_idx]
    
    # Convert to array and get the top N indices
    sorted_indices = doc.toarray()[0].argsort()[::-1]  # Sort by TF-IDF score in descending order
    
    # Extract the top N words
    top_words = [feature_names[i] for i in sorted_indices[:top_n]]
    
    # Print top keywords for each episode
    print(f"Top {top_n} keywords in episode {episodes[doc_idx]['ep_num']}: {', '.join(top_words)}")

    episodes[doc_idx]['keywords'] = top_words


Top 6 keywords in episode 6: 打電動, 小時候, 小孩, 說謊, 電視, 作業
Top 6 keywords in episode 60: 面試, 公司, 工作, 雜草, 設計, 問題
Top 6 keywords in episode 60: 蜈蚣, 蟑螂, 壁虎, 水蟻, 馬路, 尾巴
Top 6 keywords in episode 61: 牙齒, 刷牙, 牙刷, 舞龍, 舞獅, 布子
Top 6 keywords in episode 62: 遊戲, crush, candy, 抽到, 刷牙, 結牙
Top 6 keywords in episode 63: 大便, 褲子, 上廁所, t恤, 班長, 便秘
Top 6 keywords in episode 64: 咖啡廳, 告白, 天氣瓶, 南灣, msn, 室友
Top 6 keywords in episode 65: 蚊子, 葡萄, 馬蹄, 影片, 牙結屎, 遊戲
Top 6 keywords in episode 66: 吸管, 筷子, 文字, 胡椒, 菠菜, 登登
Top 6 keywords in episode 67: 阿公, 阿嬤, 奶奶, 平板, 爺爺, 步道
Top 6 keywords in episode 68: 哼哼, 哼哼哼哼, 橘子, 桃園, 整理, 丁香
Top 6 keywords in episode 69: 金門, 老師, 考卷, 學校, 蛋餅, 比賽


In [45]:
episodes[0]['keywords']

['打電動', '小時候', '小孩', '說謊', '電視', '作業']

In [48]:
for episode in episodes:
    upsert_data = {
        'keywords': episode['keywords']
    }
    result = episodes_collection.update_one(
        {
            "episode_number": episode["ep_num"],
            "episode_part": episode['ep_part']
        },  # Query by episode_number
        {"$set": upsert_data},  # Set the episode data
        upsert=True  # Insert if it does not exist
    )
    print(f"Episode {episode['ep_num']} upserted. {result.modified_count} document(s) modified.")

Episode 6 upserted. 1 document(s) modified.
Episode 60 upserted. 1 document(s) modified.
Episode 60 upserted. 1 document(s) modified.
Episode 61 upserted. 1 document(s) modified.
Episode 62 upserted. 1 document(s) modified.
Episode 63 upserted. 1 document(s) modified.
Episode 64 upserted. 1 document(s) modified.
Episode 65 upserted. 1 document(s) modified.
Episode 66 upserted. 1 document(s) modified.
Episode 67 upserted. 1 document(s) modified.
Episode 68 upserted. 1 document(s) modified.
Episode 69 upserted. 1 document(s) modified.


In [9]:
# LDA
import gensim
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel

texts = [[word for word in ep['words'] if word not in stopwords and len(word) > 1] for ep in episodes]
id2word = corpora.Dictionary(texts)

# Create a document-term matrix
corpus = [id2word.doc2bow(text) for text in texts]

# Step 3: Build the LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=42, passes=10)

# Step 4: Print the topics with the top words for each topic
for idx, topic in lda_model.print_topics(num_words=4):
    print(f"Topic {idx+1}: {topic}")

Topic 1: 0.000*"哼哼" + 0.000*"一樣" + 0.000*"發現" + 0.000*"時間"
Topic 2: 0.006*"登登" + 0.005*"如果" + 0.004*"一樣" + 0.004*"雨傘"
Topic 3: 0.005*"牙齒" + 0.005*"老師" + 0.004*"刷牙" + 0.004*"如果"
Topic 4: 0.013*"吸管" + 0.009*"筷子" + 0.006*"討厭" + 0.006*"上面"
Topic 5: 0.000*"發現" + 0.000*"一樣" + 0.000*"一點" + 0.000*"如果"
Topic 6: 0.000*"小孩" + 0.000*"一樣" + 0.000*"發現" + 0.000*"老師"
Topic 7: 0.008*"影片" + 0.006*"OK" + 0.005*"問題" + 0.005*"蚊子"
Topic 8: 0.000*"哼哼" + 0.000*"不能" + 0.000*"一樣" + 0.000*"時間"
Topic 9: 0.005*"一樣" + 0.005*"面試" + 0.005*"問題" + 0.005*"想要"
Topic 10: 0.001*"哼哼" + 0.000*"小時候" + 0.000*"如果" + 0.000*"發現"
Topic 11: 0.015*"遊戲" + 0.005*"發現" + 0.005*"刷牙" + 0.005*"一樣"
Topic 12: 0.028*"哼哼" + 0.007*"老師" + 0.006*"第一" + 0.006*"學校"
Topic 13: 0.016*"小時候" + 0.013*"小孩" + 0.009*"打電動" + 0.008*"發現"
Topic 14: 0.013*"金門" + 0.007*"印象" + 0.006*"一樣" + 0.005*"小時候"
Topic 15: 0.012*"阿公" + 0.008*"阿嬤" + 0.006*"一樣" + 0.005*"奶奶"
Topic 16: 0.009*"大便" + 0.008*"上廁所" + 0.007*"問題" + 0.006*"褲子"
Topic 17: 0.010*"蜈蚣" + 0.009*"蟑螂" + 0.006*"壁

In [10]:
# Word Cloud